Analysis of Store Time-Series Forcasting on Kaggle 
- [Found here](https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data?select=holidays_events.csv)

In [1]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


def load_file(name):
    dir = "datasets/store_forecasting/"
    return pd.read_csv(f'{dir+name}')


train = load_file("train.csv")
test = load_file("test.csv")
oil = load_file("oil.csv")
transactions = load_file("transactions.csv")
holidays = load_file("holidays_events.csv")
stores = load_file("stores.csv")

In [2]:
train.dtypes

id               int64
date            object
store_nbr        int64
family          object
sales          float64
onpromotion      int64
dtype: object

In [3]:
train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [4]:
len(train.store_nbr.unique())

54

In [5]:
stores

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15


Joining the stores to the item data
- note the "many to one" join

In [6]:
joined_store_train  = train.join(stores.set_index("store_nbr"), on="store_nbr", validate="m:1")

joined_store_train

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6


In [7]:
oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


Join on Oil

In [8]:
joined_oil_store_train = joined_store_train.join(oil.set_index("date"), on="date", validate="m:1")


print(joined_oil_store_train.state.unique())

(joined_oil_store_train)

['Pichincha' 'Cotopaxi' 'Chimborazo' 'Imbabura'
 'Santo Domingo de los Tsachilas' 'Bolivar' 'Pastaza' 'Tungurahua'
 'Guayas' 'Santa Elena' 'Los Rios' 'Azuay' 'Loja' 'El Oro' 'Esmeraldas'
 'Manabi']


,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,NaN
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,NaN
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,NaN
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,NaN
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,47.57
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,47.57
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,47.57
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,47.57


# Holidays 
- look at transferred holidays
- be note of "bridge" holidays, additional holidays given out
- take into consideration if the holiday is for thet region/locale

In [9]:
holidays_no_transfer = holidays[holidays.transferred == False]

holidays.type.unique()

np.sort(holidays_no_transfer.locale.unique())

holidays_no_transfer

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


Join the city holidays on eachother

There are 3 types of holidays:
- national holidays (all of Ecuador) (National)
- state holidys (Regional)
- city holidays (Local)

In [10]:
national_holidays = holidays_no_transfer[holidays_no_transfer['locale'] == 'National']
holidays_no_transfer.drop_duplicates(subset="date", inplace=True)

#holidays_no_transfer.columns
#pivot_holidays = holidays_no_transfer.pivot(index=["date", "locale"], columns="type", values="locale_name")

table = holidays_no_transfer.set_index(['locale', 'date'])
table = table[['description', 'locale_name']]

regional = table.loc['Regional']

local  =  table.loc['Local']

national = table.loc['National']

national


/var/folders/cv/lyp80g3175v43k438cr6bx700000gq/T/ipykernel_54196/1851474218.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holidays_no_transfer.drop_duplicates(subset="date", inplace=True)


,description,locale_name
date,,
2012-08-10,Primer Grito de Independencia,Ecuador
2012-10-12,Traslado Independencia de Guayaquil,Ecuador
2012-11-02,Dia de Difuntos,Ecuador
2012-11-03,Independencia de Cuenca,Ecuador
2012-12-21,Navidad-4,Ecuador
...,...,...
2017-12-21,Navidad-4,Ecuador
2017-12-23,Navidad-2,Ecuador
2017-12-24,Navidad-1,Ecuador


You were merging on national and then reusing that but losing the columns in the process

In [11]:


merged = joined_oil_store_train.merge(national, on="date",suffixes=('_merged', "_a"), how="left")
merged_regional = merged.merge(regional, left_on=['date', 'state'], right_on=['date' , 'locale_name'], how='left', suffixes=("_regional", "_national"))
merged_al = merged_regional.merge(local,left_on=['date', 'city'], right_on=['date', 'locale_name'], how='left', suffixes=("_city","_regional", "_national"))

merged_al

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,description_regional,locale_name_regional,description_national,locale_name_national,description,locale_name
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,NaN,Primer dia del ano,Ecuador,NaN,NaN,NaN,NaN
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,NaN,Primer dia del ano,Ecuador,NaN,NaN,NaN,NaN
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,NaN,Primer dia del ano,Ecuador,NaN,NaN,NaN,NaN
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,NaN,Primer dia del ano,Ecuador,NaN,NaN,NaN,NaN
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,NaN,Primer dia del ano,Ecuador,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,47.57,NaN,NaN,NaN,NaN,NaN,NaN
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,47.57,NaN,NaN,NaN,NaN,NaN,NaN
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,47.57,NaN,NaN,NaN,NaN,NaN,NaN
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,47.57,NaN,NaN,NaN,NaN,NaN,NaN


Make a holiday column after all this

In [12]:
merged_al['locale_name_regional'].isna() |  (merged_al['locale_name_national']).isna() | (merged_al['locale_name']).isna()


merged_al['Holiday'] =  merged_al['locale_name_regional'].notna() | (merged_al['locale_name_national']).notna() | (merged_al['locale_name']).notna()

merged_al

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,description_regional,locale_name_regional,description_national,locale_name_national,description,locale_name,Holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,NaN,Primer dia del ano,Ecuador,NaN,NaN,NaN,NaN,True
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,NaN,Primer dia del ano,Ecuador,NaN,NaN,NaN,NaN,True
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,NaN,Primer dia del ano,Ecuador,NaN,NaN,NaN,NaN,True
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,NaN,Primer dia del ano,Ecuador,NaN,NaN,NaN,NaN,True
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,NaN,Primer dia del ano,Ecuador,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,47.57,NaN,NaN,NaN,NaN,NaN,NaN,False
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,47.57,NaN,NaN,NaN,NaN,NaN,NaN,False
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,47.57,NaN,NaN,NaN,NaN,NaN,NaN,False
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,47.57,NaN,NaN,NaN,NaN,NaN,NaN,False


In [13]:
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder


# Preprocessing step
def merge_holiday_store_oil(dataset):
    
    dataset  = dataset.join(stores.set_index("store_nbr"), on="store_nbr", validate="m:1")

    dataset = dataset.join(oil.set_index("date"), on="date", validate="m:1")

    holidays_no_transfer = holidays[holidays.transferred == False]
    holidays_no_transfer.drop_duplicates(subset="date", inplace=True)
    table = holidays_no_transfer.set_index(['locale', 'date'])
    table = table[['description', 'locale_name']]
    regional = table.loc['Regional']
    local  =  table.loc['Local']
    national = table.loc['National']
    
    merged = dataset.merge(national, on="date",suffixes=('_merged', "_a"), how="left")
    merged_regional = merged.merge(regional, left_on=['date', 'state'], right_on=['date' , 'locale_name'], how='left', suffixes=("_regional", "_national"))
    merged_all = merged_regional.merge(local,left_on=['date', 'city'], right_on=['date', 'locale_name'], how='left', suffixes=("_city","_regional", "_national"))
    merged_all['Holiday'] =  merged_all['locale_name_regional'].notna() | (merged_all['locale_name_national']).notna() | (merged_all['locale_name']).notna()
    
    merged_all['date'] = pd.to_datetime(dataset['date'])
    return merged_all
     



In [14]:

train_data = merge_holiday_store_oil(train) 
y = train_data['sales']
X = train_data.drop("sales", axis=1)


/var/folders/cv/lyp80g3175v43k438cr6bx700000gq/T/ipykernel_54196/946098167.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holidays_no_transfer.drop_duplicates(subset="date", inplace=True)


In [24]:

import sklearn 

sklearn.set_config(transform_output="pandas")

numerical = make_pipeline(
    StandardScaler()
)


categorical = make_pipeline(
    OneHotEncoder(sparse_output=False, drop='if_binary')
)


columns = make_column_transformer(
    (categorical, ['family', 'type','Holiday' ]),
     (numerical, ['dcoilwtico', 'cluster'])
     , remainder="drop", verbose_feature_names_out=False
)





family_AUTOMOTIVE                    float64
family_BABY CARE                     float64
family_BEAUTY                        float64
family_BEVERAGES                     float64
family_BOOKS                         float64
family_BREAD/BAKERY                  float64
family_CELEBRATION                   float64
family_CLEANING                      float64
family_DAIRY                         float64
family_DELI                          float64
family_EGGS                          float64
family_FROZEN FOODS                  float64
family_GROCERY I                     float64
family_GROCERY II                    float64
family_HARDWARE                      float64
family_HOME AND KITCHEN I            float64
family_HOME AND KITCHEN II           float64
family_HOME APPLIANCES               float64
family_HOME CARE                     float64
family_LADIESWEAR                    float64
family_LAWN AND GARDEN               float64
family_LINGERIE                      float64
family_LIQ

In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_validate

model = HistGradientBoostingRegressor(random_state=42)

pipe = make_pipeline(columns, model)

pipe.fit(X,y)




Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 sparse_output=False))]),
                                                  ['family', 'type',
                                                   'Holiday']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['dcoilwtico', 'cluster'])],
                                   verbose_feature_names_out=False)),
                ('histgradientboostingregressor',
                 HistGradientBoostingRegressor(random_state=42))])

In [30]:
test_data = merge_holiday_store_oil(test)
y_pred = pipe.predict(test_data)

submission = pd.DataFrame()

submission['id'] = test_data['id']
submission['sales'] = y_pred

submission.to_csv("submissions/Ecuador_Stores.csv", header=True, index=False)



/var/folders/cv/lyp80g3175v43k438cr6bx700000gq/T/ipykernel_54196/946098167.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holidays_no_transfer.drop_duplicates(subset="date", inplace=True)
